In [36]:
# -----------------------------------------------
#                  LIBRARIES
# -----------------------------------------------

import geopandas as gpd 
import numpy as np
import pandas as pd 
from fiona.drvsupport import supported_drivers
import requests
import json
import datetime
import time

# ------------------------------
#       DATA TREATMENT
# ------------------------------

# IMPORT KML DRIVER

supported_drivers['KML'] = 'rw'

# READ KML file to a geopandas dataframe 

geo_df = gpd.read_file('CentrosSecundariaMadrid.kml',driver='KML')

# Create Pandas Dataframe from GeoPandas 

df= pd.DataFrame(geo_df)

# Request api-key
f = open('api_key.json',)
credentials = json.load(f)
api_key = credentials['gmaps_api_key']
ors_api_key = credentials['ors_api_key']

In [4]:
# Parámetros del script

# Tiempo de salida (Año, Mes, Día, Hora, Minuto, Segundo)

TIEMPOSALIDA = str(int(datetime.datetime(2023, 6, 7, 7, 30, 0).timestamp()))

# Modo de transporte: 'driving', 'walking', 'bicycling', 'transit'

MODODETRANSPORTE = 'driving'

direccion = input()

 Mayor 34, Algete


In [5]:
df_xy = geo_df['geometry'].get_coordinates()

In [6]:
df = pd.concat([df, df_xy], axis=1)

In [7]:
df.drop(df[df['x'] == 0].index, inplace = True)

In [8]:
coord_ies = ["{},{}".format(y,x) for x,y in zip(df['x'],df['y'])]

In [9]:
chunks = [coord_ies[i:i+25] for i in range(0, len(coord_ies), 25)]

In [37]:
#REQUEST API DIRECCION

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
}
call = requests.get('https://api.openrouteservice.org/geocode/search?api_key={}&text={}&boundary.country=ES'.format(ors_api_key,direccion), headers=headers)

print(call.status_code, call.reason)

data_dir = json.loads(call.text)

200 OK


In [11]:
data_dir['features'][0]['geometry']['coordinates']

[-3.507326, 40.59644]

In [12]:
coord_dir = data_dir['features'][0]['geometry']['coordinates']
coord_dir_str = str("{},{}".format(coord_dir[1],coord_dir[0]))

In [33]:
def call_chunk(chunk):

    # Take source
    source = coord_dir_str

    # Take destinations
    dest = '|'.join(chunk)

    # Take arrival time
    depart_time = TIEMPOSALIDA
    
    # Take mode of transport
    
    trans_mode = MODODETRANSPORTE
    
    # url variable store url
    url ='https://maps.googleapis.com/maps/api/distancematrix/json?'

    # Get method of requests module
    # return response object
    r = requests.get(url + 'origins=' + source +
                       '&destinations=' + dest +
                       '&key=' + api_key +
                       '&mode=' + trans_mode +
                       '&departure_time=' + depart_time + '&avoid=tolls')
    data = r.text
    # json method of response object
    # return json format result
    return r.json()

In [39]:
duration_list = []
for chunk in chunks[:1]:
    v = call_chunk(chunk)
    #time.sleep(0.5)
    for element in v['rows'][0]['elements']:
        if element['status'] == 'OK':
            if MODODETRANSPORTE == 'driving':
                duration = element['duration_in_traffic']['value']
            else:
                duration = element['duration']['value']
        else:
            duration = 99999
            print(element['status'])
        duration_list.append(duration)

In [41]:
len(duration_list)

25

In [17]:
df['Duraciones'] = duration_list

In [18]:
from humanfriendly import format_timespan

df['Tiempo'] = [format_timespan(i) if i != 99999 else 'No se puede llegar' for i in df['Duraciones']]

In [19]:
df_out = df.sort_values(['Duraciones'])

In [20]:
# Extracting components
df_out['D.A.T.'] = df_out['Description'].str.split('<br>').str[0].str.split(': ').str[1]
df_out['codigo distrito'] = df_out['Description'].str.split('<br>').str[1].str.split(': ').str[1]
df_out['nombre distrito'] = df_out['Description'].str.split('<br>').str[2].str.split(': ').str[1]
df_out['codigo centro'] = df_out['Description'].str.split('<br>').str[3].str.split(': ').str[1]
df_out['lengua'] = df_out['Description'].str.split('<br>').str[4].str.split(': ').str[1]
df_out['nombre centro'] = df_out['Description'].str.split('<br>').str[5].str.split('; ').str[1].str.split(': ').str[1]
df_out['distrito'] = df_out['Description'].str.split('<br>').str[5].str.split('; ').str[0].str.split(': ').str[1]

In [21]:
df_out = df_out.drop(['Description','geometry','Duraciones','codigo distrito', 'lengua', 'distrito','nombre centro'],axis=1)

In [22]:
df_out = df_out[['Tiempo', 'Name', 'nombre distrito','codigo centro','D.A.T.','x','y']]

In [23]:
df_out

,Tiempo,Name,nombre distrito,codigo centro,D.A.T.,x,y
180,1 minute and 30 seconds,IES GUSTAVO ADOLFO BECQUER,ALGETE,28040672,Madrid-Norte,-3.510539,40.600264
179,1 minute and 39 seconds,IES AL-SATT,ALGETE,28040611,Madrid-Norte,-3.509965,40.599728
509,4 minutes and 30 seconds,CP INF-PRI-SEC VILLA DE COBEÑA,COBEÑA,28002211,Madrid-Norte,-3.506465,40.569776
516,7 minutes and 10 seconds,CP INF-PRI-SEC MARTINA GARCIA,FUENTE EL SAZ DE JARAMA,28002853,Madrid-Norte,-3.515641,40.629684
455,8 minutes and 39 seconds,CP INF-PRI-SEC SAN BLAS,AJALVIR,28000029,Madrid-Este,-3.486152,40.539876
...,...,...,...,...,...,...,...
639,"1 hour, 22 minutes and 30 seconds",EOEP ATEN-TEMPR EOEP DE AT.TNA,SAN MARTIN DE VALDEIGLESIAS,28701019,Madrid-Sur,-4.397997,40.363821
327,"1 hour, 23 minutes and 1 second",IES MAESTRO JUAN MARIA LEONET,VILLA DEL PRADO,28046273,Madrid-Sur,-4.304016,40.272253
640,"1 hour, 23 minutes and 2 seconds",EOEP GENERAL EQUIPO GENERAL S.MARTÍN,SAN MARTIN DE VALDEIGLESIAS,28700441,Madrid-Sur,-4.403503,40.363562
580,"1 hour, 29 minutes and 55 seconds",CP INF-PRI-SEC EUGENIO MURO,CADALSO DE LOS VIDRIOS,28002026,Madrid-Sur,-4.452356,40.302569


In [24]:
df_out.to_csv("Resultados/tiempos_gmaps_{}_{}.csv".format(MODODETRANSPORTE,direccion),index=False)